In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
# plt.style.use('dark_background')
sns.set()
sns.set_palette("tab10")
sns.set_context("notebook", font_scale=1)

In [ ]:
from analysis.paths import path_CESM
from analysis.paths import path_ocn_ctrl, path_ocn_ctrl_rect
from analysis.paths import path_ocn_rcp,  path_ocn_rcp_rect
from analysis.paths import file_ex_ocn_hires, file_ex_ocn_rect

In [ ]:
from operations.grid import generate_lats_lons_CESM

In [ ]:
# from binary file
grid_file  = f'{path_CESM}/inputdata/ocn/pop/tx0.1v2/grid/horiz_grid_200709.ieeer8'
imt,jmt,km = 3600, 2400, 42
lats,lons  = generate_lats_lons_CESM(grid_file)

In [ ]:
f, ax = plt.subplots(1,2, sharey=True)
ax[0].imshow(lats)
ax[1].imshow(lons)

In [ ]:
from operations.xr_integrate import generate_xr_DZ, generate_xr_AREA, create_xr_DataArray
DZT = generate_xr_DZ()
TAREA = generate_xr_AREA()
D, C, imt, jmt, km = create_xr_DataArray(file_ex_ocn_hires, n=3, fill=1)

In [ ]:
from operations.xr_integrate import xr_vol_int, xr_surf_int
print(f'The volument of the ocean is {xr_vol_int(D, TAREA, DZT)/1e9:.2e} km^3.')
print(f'The surface of the ocean is {xr_surf_int(D[0,:,:], TAREA)/1e6:.2e} km^2.')

In [ ]:
plt.imshow(DZT[:,1000,:], aspect='auto')
plt.colorbar()

In [ ]:
plt.imshow(TAREA, cmap='viridis', origin='lowerleft')
plt.colorbar()
plt.tight_layout()

In [ ]:
from analysis.paths import file_ex_ocn_hires

In [ ]:
C = xr.open_dataset(file_ex_ocn_hires, decode_times=False)

In [ ]:
plt.imshow(C.KMT, origin='lowerleft')
plt.colorbar()
plt.tight_layout()

In [ ]:
plt.imshow(C.REGION_MASK, cmap='tab20', origin='lowerleft', vmin=-1.1, vmax=18)
plt.colorbar()
plt.tight_layout()

In [ ]:
np.unique(C.REGION_MASK.values)

In [ ]:
from regions import regions
print(regions)

In [ ]:
%%time
for i, key in enumerate(regions):
    surf = xr_surf_int(D[0,:,:], TAREA.where(C.REGION_MASK==key)     )/1e6
    vol  = xr_vol_int( D       , TAREA.where(C.REGION_MASK==key), DZT)/1e9
    print(f'{i:2} {key:4} {regions[key]:16}: {surf:10.2e} km^2, {vol:10.2e} km^3')